In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
import numpy as np

# Options

In [ ]:
polarsGNVP3 = True
pertrAnalysisGNVP3 = True
sensAnalysisGNVP3 = False

# Load Foil Data

In [ ]:
from ICARUS.Software.XFLR5.polars import readPolars2D
from ICARUS.Database.db import DB
from ICARUS.Database import XFLRDB

In [ ]:
db = DB()
foildb = db.foilsDB
foildb.loadData()
readPolars2D(foildb,XFLRDB)
airfoils = foildb.getAirfoils()

# Get Plane

In [ ]:
from Data.Planes.hermes import hermes

In [ ]:
ap = hermes(airfoils ,"Hemes_HRes")

# Import Enviroment

In [ ]:
from ICARUS.Enviroment.definition import EARTH
print(EARTH)

# Get Solver

In [ ]:
from ICARUS.Software.GenuVP3.gnvp3 import get_gnvp3
gnvp3 = get_gnvp3(db)
print(gnvp3)

## AoA Run

In [ ]:
analysis = gnvp3.getAvailableAnalyses()[2] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.setAnalysis(analysis)
options = gnvp3.getOptions(analysis)

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)
Uinf = 20
ap.defineSim(Uinf, EARTH.AirDensity)

options.plane.value         = ap
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'Xfoil'
options.maxiter.value       = 200
options.timestep.value      = 5e-3
options.Uinf.value          = Uinf
options.angles.value        = angles

gnvp3.printOptions()
gnvp3.run()
ap.save()

In [ ]:
results = gnvp3.getAvailableRetrieval()[0] # ANGLES PARALLEL
print(f"Selecting Retrieval: {results}")
gnvp3.setRetrivalMethod(results)
options = gnvp3.getRetrievalOptions(results)

In [ ]:
options.HOMEDIR.value = db.analysesDB.HOMEDIR
import os
options.CASEDIR.value = os.path.join(db.vehiclesDB.DATADIR, ap.CASEDIR)
gnvp3.printRetrievalOptions()
polars = gnvp3.getResults()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.Flight_Dynamics.dyn_plane import dyn_Airplane as dp
dyn = dp(ap,polars)

### Pertrubations

In [ ]:
dyn.allPerturb("Central")
# dyn.get_pertrub()

In [ ]:
analysis = gnvp3.getAvailableAnalyses()[4] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.setAnalysis(analysis)
options = gnvp3.getOptions(analysis)

In [ ]:
dyn.defineSim(dyn.trim['U'], EARTH.AirDensity)

options.plane.value         = dyn
options.environment.value   = EARTH
options.db.value            = db
options.solver2D.value      = 'Xfoil'
options.maxiter.value       = 200
options.timestep.value      = 5e-3
options.Uinf.value          = dyn.trim['U']
options.angles.value        = dyn.trim['AoA']

gnvp3.printOptions()
gnvp3.run()
dyn.save()

In [ ]:
results = gnvp3.getAvailableRetrieval()[1] # Pertr
print(f"Selecting Retrieval: {results}")
gnvp3.setRetrivalMethod(results)
options = gnvp3.getRetrievalOptions(results)

In [ ]:
options.HOMEDIR.value = db.analysesDB.HOMEDIR
options.DYNDIR.value = os.path.join(db.vehiclesDB.DATADIR, ap.CASEDIR, "Dynamics")
gnvp3.printRetrievalOptions()
polars = gnvp3.getResults()

# Sensitivity Analysis

In [ ]:
# if sensAnalysisGNVP3 == True:
#     for var in ['u','w','q','theta','v','p','r','phi']:
#         space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
#         space =[*-space,*space]
#         maxiter = 2
#         timestep = 5e-2
#         dyn.sensitivityAnalysis(var, space)
#         GNVP3BatchArgs = [dyn, var, foildb, "Xfoil",
#                         maxiter, timestep,
#                         dyn.trim['U'], dyn.trim['AoA']]
#         dyn.runAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
#         perDIR = os.path.join(dyn.CASEDIR, f"Sensitivity_{var}")
#         dyn.sensResults[var] = pertrResults(perDIR, HOMEDIR)
#     dyn.save()